# Beter werken met functies

:::{admonition} Leerdoelen
:class: tip
In deze les staat **werken met functies in Python** centraal.

We gebruiken geluid als context. Het doel is **niet** om audioverwerking te leren, maar om:

* functies te schrijven met duidelijke parameters en returnwaarden;
* functies te combineren;
* eigen functies te bundelen in een **herbruikbare library**.
:::

Daarom plaatsen we **alle functies die we schrijven in een afzonderlijk bestand**:
`audio_functies.py`.

In het oefenbestand zelf gebruiken we die functies enkel nog via `import`.

## Een sinusgolf genereren

### Wat is een sinusgolf?

Een zuivere toon (zoals van een stemvork) kan wiskundig beschreven worden met een sinusfunctie.

De formule is:

$
y(t) = A \cdot \sin(2\pi f t)
$

waarbij:

* **( f )** de frequentie is (in Hz, trillingen per seconde)
* **( A )** de amplitude is (hoe luid het geluid is)
* **( t )** de tijd is (in seconden)

---

### Van continue formule naar digitale audio

Een computer werkt niet met een continue tijd ( t ), maar met **samples**:

* we meten het signaal op vaste tijdstippen
* het aantal metingen per seconde heet de **samplerate**

In deze les gebruiken we **altijd dezelfde samplerate**:

```python
SAMPLERATE = 16000  # samples per seconde
```

Dat betekent:

* 1 seconde audio → 16 000 samples
* 0,5 seconde audio → 8 000 samples

Door deze waarde vast te leggen als **constante**, hoeven we:

* ze niet telkens als argument mee te geven;
* ze niet telkens opnieuw te onthouden.

---

### Parameters van een toon

Bij het genereren van een toon hebben we vier zaken nodig:

* **frequentie** (Hz)
  bepaalt hoe hoog of laag de toon klinkt
* **duur** (seconden)
  hoe lang de toon duurt
* **amplitude** (meestal tussen 0 en 1)
  hoe luid de toon is
* **samplerate**
  vast ingesteld op 16000

## De eerste functie: `sine_wave`

Je plaatst deze functie in het bestand **`audio_functies.py`**.

```python
import numpy as np

SAMPLERATE = 16000

def sine_wave(freq, duration, amplitude=0.5):
    """
    Genereer een sinusgolf.

    freq       : frequentie in Hz
    duration   : duur in seconden
    amplitude  : volume (0.0 – 1.0)
    """
    t = np.linspace(
        0,
        duration,
        int(SAMPLERATE * duration),
        endpoint=False
    )

    y = amplitude * np.sin(2 * np.pi * freq * t)
    return y
```

### Wat gebeurt hier stap voor stap?

1. We maken een **tijd-as** `t`:

   * van 0 tot `duration`
   * met exact `samplerate × duration` punten

2. We passen de sinusformule toe op **alle tijdstippen tegelijk**
   (NumPy rekent dit efficiënt uit).

3. De functie **returnt een array** met samples:

   * geen geluid
   * geen plot
   * alleen data

Dat maakt de functie flexibel en herbruikbaar.

## Eerste oefening: een toon afspelen

In het **oefenbestand** (bijvoorbeeld `oefening_01.py`) schrijven we zelf **geen functies**.
We gebruiken enkel wat in `audio_functies.py` staat.

### Benodigde library installeren

```bash
pip install sounddevice numpy
```

### Code van de oefening

```python
import sounddevice as sd
import audio_functies as af

toon = af.sine_wave(freq=440, duration=1.0)

sd.play(toon, af.SAMPLERATE)
sd.wait()
```

### Wat leren we hier?

* een functie **produceert data**
* die data kunnen we:

  * afspelen
  * later bewerken
  * later visualiseren
* de naam van het argument (`freq`) staat los van de naam van de variabele (`toon`)

---

## Een golf visualiseren

### Waarom tonen we niet de hele golf?

Een sinus van 440 Hz trilt:

* **440 keer per seconde**
* bij 16 000 samples per seconde

Als we **1 seconde** plotten, zien we:

* 16 000 punten
* honderden trillingen boven elkaar

Dat wordt een dikke, onleesbare band.

Daarom tonen we **slechts een klein tijdsvenster**, bijvoorbeeld 10 of 20 milliseconden.

---

## Functie om een stukje van een waveform te tonen

Ook deze functie plaatsen we in **`audio_functies.py`**.

```python
import matplotlib.pyplot as plt
import numpy as np

def plot_wave_segment(y, t0=0.0, t1=0.02, title="Waveform"):
    """
    Toon een kort deel van een waveform.

    y  : array met samples
    t0 : starttijd (s)
    t1 : eindtijd (s)
    """
    i0 = int(t0 * SAMPLERATE)
    i1 = int(t1 * SAMPLERATE)

    segment = y[i0:i1]
    t = np.linspace(t0, t1, len(segment), endpoint=False)

    plt.figure(figsize=(8, 3))
    plt.plot(t, segment)
    plt.xlabel("tijd (s)")
    plt.ylabel("amplitude")
    plt.title(title)
    plt.grid(True)
    plt.tight_layout()
    plt.show()
```

### Wat doet deze functie?

* zet tijd (in seconden) om naar indices
* snijdt een stuk uit de golf
* maakt zelf een correcte tijd-as
* zorgt voor een nette, leesbare plot

Zonder deze functie zouden we dit **telkens opnieuw** moeten uitschrijven.

---

## Gebruik in de oefening

```python
import audio_functies as af
import sounddevice as sd

toon = af.sine_wave(freq=440, duration=1.0)

sd.play(toon, af.SAMPLERATE)
af.plot_wave_segment(toon)
sd.wait()
```

Je ziet nu:

* de vorm van de sinus
* hoe amplitude en frequentie zich visueel gedragen




## Niveau 1 — basisbewerkingen met waves

In dit deel bewerken we een waveform (een NumPy-array met samples). Het kernidee blijft:

* een waveform is een lijst getallen (samples)
* bewerkingen zijn meestal **rekenkundige bewerkingen op alle samples**
* je schrijft elke bewerking als **een functie in `audio_functies.py`**

### 1) Hoe maak ik een toon luider of zachter?

#### Concept

Een toon luider of zachter maken is een **amplitudeschaling**: je vermenigvuldigt elke sample met een factor.

Als `y` de originele wave is, dan is de aangepaste wave:

$
y_{\text{nieuw}}[n] = k \cdot y[n]
$

* (k > 1) → luider
* (0 < k < 1) → zachter
* (k = 0) → stilte

#### Wat moet je functie doen?

* input: `y` en een `factor`
* output: een **nieuwe** array met de geschaalde samples

#### Testidee in je oefenbestand

```python
import sounddevice as sd
import audio_functies as af

toon = af.sine_wave(440, 1.0)

zacht = ...   # jouw functie gebruiken
luid  = ...   # jouw functie gebruiken

sd.play(zacht, af.SAMPLERATE); sd.wait()
sd.play(luid, af.SAMPLERATE); sd.wait()
```


### 2) Hoe maak ik stilte?

#### Concept

Stilte is gewoon een wave met overal **0**.

Als je `N` samples nodig hebt:

$
y[n] = 0 \quad \text{voor alle } n
$

En:
$
N = \text{SAMPLERATE} \cdot \text{duration}
$

#### Wat moet je functie doen?

* input: `duration` (seconden)
* output: een array van lengte `int(SAMPLERATE * duration)` gevuld met nullen

#### Testidee

Speel eerst 0.5s toon, dan 0.3s stilte, dan opnieuw toon (door te concatenaten; zie vraag 4).


### 3) Hoe laat ik twee tonen samenklinken? (mixen) + clipping

#### Concept

Twee tonen samenklinken betekent: **samples optellen** (superpositie).

Als `y1` en `y2` even lang zijn:

$
y_{\text{mix}}[n] = y_1[n] + y_2[n]
$

Je kan ook mengen met verhoudingen:

$
y_{\text{mix}}[n] = a \cdot y_1[n] + b \cdot y_2[n]
$

Vaak kies je (a) en (b) zo dat het resultaat niet te luid wordt.

:::{admonition} Clipping (belangrijk!)
:class: seealso

De meeste computers verwachten samples in ongeveer het bereik **[-1, 1]** wanneer ze audio afspelen.

Als je twee tonen optelt, kan de amplitude groter worden dan 1:

* voorbeeld: als beide tonen amplitude 0.8 hebben, dan kan de som richting 1.6 gaan

Wanneer waarden “buiten het bereik” vallen, ontstaat **clipping**:

* de topjes worden “afgeknipt”
* dit klinkt als vervorming/kraken
:::

#### Wat moet je mix-functie doen?

* tel `y1 + y2` op (als ze even lang zijn)
* gebruik een andere functie om tonen zachter te maken voordat je ze doorgeeft aan deze functie

#### Testidee

* Maak twee sinussen: 440 Hz en 660 Hz
* Mix ze
* Plot een kort segment: je ziet dat de vorm ingewikkelder wordt
* Speel af: je hoort een “rijkere” klank

### 4) Hoe speel ik meerdere tonen na mekaar?

#### Concept

“Na elkaar” betekent: arrays achter elkaar plakken (**concatenation**).

Als je `y1` en `y2` achter elkaar wil:

$
y_{\text{seq}} = [y_1, y_2]
$

Je schrijft hier best een helperfunctie voor, zodat melodieën makkelijk worden. Stop al je genereerde waves in een lijst. De functie moet als argument de lijst krijgen en gebruikt dan numpy met de functie concatenate om alle elementen uit de lijst aan mekaar te plakken.

#### Wat moet je functie doen?

* input: een lijst van waves
* output: één lange wave

#### Voorbeeld van hoe je dit zou gebruiken

```python
import audio_functies as af
import sounddevice as sd
import numpy as np

notes = []
notes.append(af.sine_wave(freq=392, duration=1.0))
notes.append(af.sine_wave(freq=440, duration=1.0))
notes.append(af.sine_wave(freq=494, duration=1.0))
notes.append(af.sine_wave(freq=523, duration=1.0))
melody = af.sequence(notes) # vervang de naam door jouw functie

sd.play(melody, af.SAMPLERATE)
sd.wait()
```


## Niveau 2 — Envelopes (amplitude “vorm” over tijd)

Een **envelope** is een curve die bepaalt hoe luid een geluid is op elk moment. Je maakt een envelope als een array met evenveel samples als je waveform, en je past die toe door:

$
y_{\text{nieuw}}[n] = y[n] \cdot e[n]
$

waarbij `e[n]` typisch tussen 0 en 1 ligt.

Praktisch: je maakt `e` met NumPy en vermenigvuldigt elementgewijs.

### 1) Hoe maak ik een fade-in en fade-out?

#### Fade-in (lineair)

Een fade-in is een envelope die start op 0 en geleidelijk naar 1 gaat.

**Code binnen de functie (zonder def):**

```python
# bereken hoeveel samples de fade nodig heeft
n = len(y)
fade_samples = int(af.SAMPLERATE * fade_duration)
fade_samples = min(fade_samples, n)  # veiligheid

# maak een lijst die even lang is als je wave, gevuld met '1'
e = np.ones(n)

# vervang het fade-in gedeelte door waarden van 0 tot 1
e[:fade_samples] = np.linspace(0.0, 1.0, fade_samples, endpoint=False)

# gebruik de nieuwe lijst om de amplitude van y te wijzigen
y_new = y * e
return y_new
```

#### Fade-out (lineair)

Een fade-out is het omgekeerde: van 1 naar 0 op het einde.

```python
n = len(y)
fade_samples = int(af.SAMPLERATE * fade_duration)
fade_samples = min(fade_samples, n)

e = np.ones(n)

# lineaire fade-out van 1 → 0
e[-fade_samples:] = np.linspace(1.0, 0.0, fade_samples, endpoint=False)

y_new = y * e
return y_new
```

#### Fade-in én fade-out samen

Vaak wil je beide: start zonder klik, stop zonder klik.

```python
n = len(y)
fade_samples = int(af.SAMPLERATE * fade_duration)
fade_samples = min(fade_samples, n // 2)  # zodat in/out niet overlappen

e = np.ones(n)

e[:fade_samples] = np.linspace(0.0, 1.0, fade_samples, endpoint=False)
e[-fade_samples:] = np.linspace(1.0, 0.0, fade_samples, endpoint=False)

y_new = y * e
return y_new
```

:::{admonition} **Waarom `n//2`?**
:class: seealso
Als je waveform heel kort is, mogen fade-in en fade-out niet “door elkaar” lopen.
:::

### 2) Hoe start ik met een sterke aanzet en daarna snel zachter?

Dit is een klassieke envelope: een **snelle attack** gevolgd door een **snelle decay** (en eventueel een sustain). Je kan dit heel eenvoudig maken met een **piecewise envelope**: je plakt meerdere stukjes curve na elkaar.

* attack: snel van 0 → 1 (bv. 10 ms)
* decay: daarna snel van 1 → lager niveau (bv. 0.3) (bv. 150 ms)
* rest: constant op sustain-level (0.3)

**Code binnen de functie:**

```python
n = len(y)

attack = int(af.SAMPLERATE * attack_s)
decay  = int(af.SAMPLERATE * decay_s)

attack = max(1, min(attack, n))
decay  = max(1, min(decay, n - attack))

sustain_level = 0.3

e = np.ones(n) * sustain_level

# attack: 0 → 1
e[:attack] = np.linspace(0.0, 1.0, attack, endpoint=False)

# decay: 1 → sustain_level
e[attack:attack+decay] = np.linspace(1.0, sustain_level, decay, endpoint=False)

y_new = y * e
return y_new
```

Dit geeft een duidelijke “pluk”-achtige aanzet.

## Niveau 3

### 1) Hoe kies ik de juiste frequenties?

:::{admonition} Probleem
:class: seealso

Tot nu toe schreven we frequenties rechtstreeks (440, 660, …). Dat werkt, maar is onhandig als je echt melodieën wil maken.

Muziek gebruikt noten die samen een klavier vormen. In plaats van noten als namen (“A4”, “C5”) gebruiken we hier een **nummering**: **MIDI-noten**.
:::

### MIDI in één zin

Een piano kan je zien als een reeks toetsen met nummers.
In MIDI is **noot 69** afgesproken als **A4 = 440 Hz**.

Vanaf daar geldt:

* 12 toetsen hoger = één octaaf hoger = frequentie × 2
* 12 toetsen lager = één octaaf lager = frequentie ÷ 2

Dus elke stap van 1 MIDI-nummer is een vermenigvuldiging met dezelfde factor.

#### Formule

$
f = 440 \cdot 2^{\frac{m - 69}{12}}
$

waarbij:

* (m) de MIDI-noot is (een geheel getal)
* (f) de frequentie in Hz

#### Code (in `audio_functies.py`)

**(Hier mag je dit exact overnemen als functie.)**

```python

def midi_to_freq(m):
    """
    Zet een MIDI-nootnummer om naar een frequentie in Hz.
    m = 69 komt overeen met A4 = 440 Hz.
    """
    return 440.0 * (2.0 ** ((m - 69) / 12))
```

#### Hoe werkt deze functie?

* `m - 69` zegt hoeveel toetsen je van A4 verwijderd bent
* delen door 12 zet “aantal toetsen” om naar “aantal octaven”
* `2 ** (octaven)` is de factor waarmee de frequentie verandert
* maal 440 geeft de echte frequentie

**Voorbeelden (ter controle):**

* `midi_to_freq(69)` → 440 Hz
* `midi_to_freq(81)` → 880 Hz (12 hoger = dubbel)
* `midi_to_freq(57)` → 220 Hz (12 lager = half)

#### Waarom is dit handig?

Je kan nu melodieën schrijven als nummers:

* `[69, 71, 72, 71]` in plaats van `[440, 494, 523, 494]`

### 2) Hoe maak ik een instrument?

:::{admonition} Probleem
:class: seealso

Een zuivere sinus klinkt “saai” en kunstmatig. Echte instrumenten hebben een rijkere klank, omdat ze niet alleen de grondtoon bevatten, maar ook **boventonen** (harmonics).
:::

#### Idee: klank = som van sinussen

We nemen een basisfrequentie (f) en voegen een paar sinussen toe:

* grondtoon: (f)
* 2e boventoon: (2f)
* 3e boventoon: (3f)
* …

Met dalende amplitudes (hoe hoger, hoe zachter):

$
y(t) = a_1 \sin(2\pi f t) + a_2 \sin(2\pi (2f) t) + a_3 \sin(2\pi (3f) t) + \dots
$

Typisch:

* (a_1 = 1.0)
* (a_2 = 0.5)
* (a_3 = 0.25)
* …

Dit is een heel eenvoudig “additief synthesemodel”, maar het klinkt meteen herkenbaarder.

#### Stap A — meerdere sinussen genereren en optellen

Je kiest bijvoorbeeld 4 componenten.

Conceptueel:

* maak sine op (f)
* maak sine op (2f)
* maak sine op (3f)
* maak sine op (4f)
* mix ze met dalende gewichten

Praktisch punt:

* na het optellen kan het signaal te luid worden → normaliseren is nuttig.

#### Stap B — envelope toepassen (van niveau 2)

Echte instrumenten beginnen niet “plots” en eindigen niet “hard”. Je gebruikt dus een envelope:

* korte attack (snelle aanzet)
* decay naar lager niveau
* eventueel fade-out

Je doet dit door te vermenigvuldigen:

$
y_{\text{instrument}}[n] = y_{\text{som}}[n] \cdot e[n]
$

waar `e[n]` je envelope-array is.

#### Stap C
Geef nu de uiteindelijke wave als resultaat van je functie.
